In [ ]:
# Part 1: LLaMA Base Model

# Import necessary libraries
import torch
import wandb
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments
from datasets import load_dataset, Dataset
import pandas as pd
import os

In [ ]:

# Step 1: Initialize W&B for logging
wandb.init(project="hw8_llama_base_model", name="llama_base_experiment")

In [ ]:
# Step 2: Data Preprocessing (Reusing HW7 pipeline)
def load_and_preprocess_data():
    dataset_path = "/content/drive/MyDrive/hw7_dataset.csv"  # Update with actual path
    data = pd.read_csv(dataset_path)
    data["processed_text"] = data["text"].apply(lambda x: x.lower().strip())
    return data

data = load_and_preprocess_data()

# Convert data to Hugging Face Dataset format
def prepare_hf_dataset(data):
    dataset = Dataset.from_pandas(data)
    return dataset

hf_dataset = prepare_hf_dataset(data)

In [ ]:
# Step 3: Model Initialization
# Define a function to load the LLaMA base model
def initialize_base_model(model_name):
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForCausalLM.from_pretrained(model_name)
    return tokenizer, model

In [ ]:
# LLaMA Base Model
base_model_name = "meta-llama/LLaMA-base"
base_tokenizer, base_model = initialize_base_model(base_model_name)

In [ ]:

# Step 4: Define Training Arguments
training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/hw8_llama_base_results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="/content/drive/MyDrive/hw8_llama_base_logs",
    report_to="wandb",
    logging_steps=10,
    save_total_limit=2
)


In [ ]:
# Step 5: Define Trainer
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = logits.argmax(axis=-1)
    accuracy = (predictions == labels).mean()
    return {"accuracy": accuracy}

In [ ]:
trainer = Trainer(
    model=base_model,
    args=training_args,
    train_dataset=hf_dataset,
    eval_dataset=hf_dataset,
    tokenizer=base_tokenizer,
    compute_metrics=compute_metrics
)

In [ ]:
# Step 6: Train and Evaluate
def train_and_evaluate():
    trainer.train()
    trainer.evaluate()

train_and_evaluate()


In [ ]:
# Step 7: Test Zero-Shot Classification
sample_text = "The movie was thrilling and engaging."
def zero_shot_classification(text, tokenizer, model):
    inputs = tokenizer(f"Classify: {text}", return_tensors="pt")
    outputs = model.generate(**inputs)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

zero_shot_result = zero_shot_classification(sample_text, base_tokenizer, base_model)
print("Zero-Shot Classification Result (Base Model):", zero_shot_result)

In [ ]:
# Finalize W&B
wandb.finish()
